In [1]:
# !pip install -U lightautoml
# !pip install pandas==1.4.3

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e10/sample_submission.csv
/kaggle/input/playground-series-s4e10/train.csv
/kaggle/input/playground-series-s4e10/test.csv
/kaggle/input/loan-approval-prediction/credit_risk_dataset.csv


In [3]:
X_TRAIN = pd.read_csv('/kaggle/input/playground-series-s4e10/train.csv')
X_TEST = pd.read_csv('/kaggle/input/playground-series-s4e10/test.csv')
X_ORIG = pd.read_csv('/kaggle/input/loan-approval-prediction/credit_risk_dataset.csv')

In [4]:
X_TRAIN = pd.concat([X_TRAIN, X_ORIG])

In [5]:
X_TRAIN['person_emp_length'] = X_TRAIN['person_emp_length'].fillna(X_TRAIN['person_emp_length'].median())
X_TRAIN['loan_int_rate'] = X_TRAIN['loan_int_rate'].fillna(X_TRAIN['loan_int_rate'].median())

In [6]:
X_TEST.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39098 entries, 0 to 39097
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   id                          39098 non-null  int64  
 1   person_age                  39098 non-null  int64  
 2   person_income               39098 non-null  int64  
 3   person_home_ownership       39098 non-null  object 
 4   person_emp_length           39098 non-null  float64
 5   loan_intent                 39098 non-null  object 
 6   loan_grade                  39098 non-null  object 
 7   loan_amnt                   39098 non-null  int64  
 8   loan_int_rate               39098 non-null  float64
 9   loan_percent_income         39098 non-null  float64
 10  cb_person_default_on_file   39098 non-null  object 
 11  cb_person_cred_hist_length  39098 non-null  int64  
dtypes: float64(3), int64(5), object(4)
memory usage: 3.6+ MB


In [7]:
X_TRAIN.pop('id')
id = X_TEST.pop('id')
Y_TRAIN = X_TRAIN.pop('loan_status')

In [8]:
numerical = X_TRAIN.select_dtypes(['int64', 'float64']).columns.to_list()
categorical = X_TRAIN.select_dtypes(['object']).columns.to_list()

In [9]:
print('numerical ', numerical)
print('categorical', categorical)

numerical  ['person_age', 'person_income', 'person_emp_length', 'loan_amnt', 'loan_int_rate', 'loan_percent_income', 'cb_person_cred_hist_length']
categorical ['person_home_ownership', 'loan_intent', 'loan_grade', 'cb_person_default_on_file']


In [10]:
X_TRAIN_COPY = X_TRAIN.copy()
X_TEST_COPY = X_TEST.copy()

X_TRAIN_COPY2 = X_TRAIN.copy()
X_TEST_COPY2 = X_TEST.copy()
categorical = X_TRAIN.select_dtypes(['object']).columns.to_list()
for col in categorical:
    X_TRAIN_COPY2[col] = X_TRAIN_COPY2[col].astype('category')
    X_TEST_COPY2[col] = X_TEST_COPY2[col].astype('category')

In [11]:
X_TRAIN_COPY2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 91226 entries, 0 to 32580
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype   
---  ------                      --------------  -----   
 0   person_age                  91226 non-null  int64   
 1   person_income               91226 non-null  int64   
 2   person_home_ownership       91226 non-null  category
 3   person_emp_length           91226 non-null  float64 
 4   loan_intent                 91226 non-null  category
 5   loan_grade                  91226 non-null  category
 6   loan_amnt                   91226 non-null  int64   
 7   loan_int_rate               91226 non-null  float64 
 8   loan_percent_income         91226 non-null  float64 
 9   cb_person_default_on_file   91226 non-null  category
 10  cb_person_cred_hist_length  91226 non-null  int64   
dtypes: category(4), float64(3), int64(4)
memory usage: 5.9 MB


In [12]:
def factorizeCategoricalColumns(data):
    dataCopy = data.copy()
    categorical = data.select_dtypes(['object']).columns.to_list()
    for col in categorical:
        dataCopy[col], _ = dataCopy[col].factorize(sort=True)
    return dataCopy

X_TRAIN_COPY = factorizeCategoricalColumns(X_TRAIN_COPY)
X_TEST_COPY = factorizeCategoricalColumns(X_TEST_COPY)

In [13]:
# oe = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
# X_oe = pd.DataFrame(oe.fit_transform(X[cat_cols]), columns = cat_cols).fillna(0).astype(int)

# test_oe = pd.DataFrame(oe.transform(test_data[cat_cols]), columns = cat_cols).fillna(0).astype(int)
# X_oe_concated = pd.concat([X_oe, X[num_cols]], axis= 1)
# test_oe_concated = pd.concat([test_oe, test_data[num_cols]], axis= 1)

In [14]:
for col in X_TRAIN.columns.to_list():
    X_TRAIN[col] = X_TRAIN[col].astype('string')
    X_TEST[col] = X_TEST[col].astype('string')

In [15]:
#X_TRAIN.shape

lgb_params= {
    'objective': 'binary',
    'metric': 'auc',
    'max_depth': -1,
    'verbosity': -1,
    'n_estimators': 1500,
    'max_bin': 1024,
    'boosting_type': 'gbdt', #'dart'
    'colsample_bytree': 0.5673775386473462,        
    'eta': 0.05446876730023387,
    'reg_lambda': 10.787843597294561,
    'min_child_samples': 69,
    'random_state': 42,
    #'early_stopping_rounds': 150,
    'verbose':1,
    #'categorical_feature': cat_indices,
}

xgb_params= {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'max_depth': 7,
    'eta': 0.07964177396162775,
    'reg_lambda': 38.499443612904315,
    'subsample': 0.8778759317150353,
    'colsample_bytree': 0.6504220261795185,
    'random_state': 42,
    'verbosity':0,
    'enable_categorical': True,
    'min_child_weight': 5,
    #'tree_method': 'hist',
}
xgb_params1 = {
    'eval_metric': 'auc',
    'n_estimators': 5000,
    'learning_rate': 0.10999995057648126,
    'max_depth': 8,
    'subsample': 0.9999999999999999,
    'colsample_bytree': 0.49681888251576845,
    'reg_alpha': 8.919028743833996e-06,
    'reg_lambda': 9.999999999999998,
    'gamma': 0.09787383664032377,
    'min_child_weight': 1.0,
    'max_bin': 262143,
#     'tree_method': 'hist',
#     'device': 'cuda',
    'enable_categorical': True,
    #'early_stopping_rounds': 100,
}

cb_params_3 = {'task_type': "CPU",
'iterations': 891,
'depth': 7,
'learning_rate': 0.0448563477253477,
'l2_leaf_reg': 8.309884320742215,
'border_count': 161,
'random_strength': 5.297381835241815,
'bagging_temperature': 0.10498408144882451,
'scale_pos_weight': 2.3753101495123747,
'verbose': False}

gb_best_params = {'random_state': 42,
                  'learning_rate': 0.08556786152814144, 
                  'max_depth': 11, 'n_estimators': 1130, 
                  'max_leaf_nodes': 90, 
                  'min_samples_leaf': 0.0025834075077401774, 
                  'min_samples_split': 0.11732169002596993, 
                  'min_weight_fraction_leaf': 0.0014666658781746856, 
                  'subsample': 0.9514006946730825}

hgb_params = {
    'categorical_features': categorical,
    'l2_regularization': 3.2797348813056515,
    'learning_rate': 0.04290001508632896,
    'max_depth': 292,
    #'max_features': 0.9605841946578942,
    'max_iter': 2978,
    'max_leaf_nodes': 38,
    'min_samples_leaf': 254,
    'random_state': 42
}

lgb_dart_params = {
    'boosting_type': 'dart',
    'colsample_bytree': 0.46939540409784575,
    'learning_rate': 0.0877911638744417,
    'min_child_samples': 61,
    'min_child_weight': 0.7580677713575383,
    'n_estimators': 877,
    'n_jobs': -1,
    'num_leaves': 35,
    'random_state': 1859,
    'reg_alpha': 2.5638237507752457,
    'reg_lambda': 1.8975230326974724,
    'scale_pos_weight': 2.7023753131827455,
    'subsample': 0.8009452917870442,
    'verbose': -1
}

lgb_goss_params = {
    'boosting_type': 'goss',
    'colsample_bytree': 0.24954643970263424,
    'learning_rate': 0.07188725215845243,
    'max_bins': 5626,
    'min_child_samples': 22,
    'min_child_weight': 0.7179252102190649,
    'n_estimators': 1319,
    'n_jobs': -1,
    'num_leaves': 21,
    'random_state': 42,
    'reg_alpha': 6.047743594793198,
    'reg_lambda': 5.277073776908381,
    'scale_pos_weight': 5.885539605359165,
    'subsample': 0.16046944461130142,
    'verbose': -1
}

cb_params_4 ={'objective': 'Logloss',
                                   'eval_metric': "AUC",
                                   'verbose': 0,
                                   'random_state': 42,
                                   'n_estimators': 5000,
                                   'scale_pos_weight': 2.3753101495123747,
                                   'learning_rate': 0.08746007467982597,
                                   'l2_leaf_reg': 3.50212953568363,
                                   'bagging_temperature': 0.15909129439715475,
                                   'random_strength': 1.6930893184821436,
                                   'depth': 4, 
                                   'min_data_in_leaf': 203,
                                   'grow_policy': 'SymmetricTree',
                                   'boosting_type': 'Plain',
                                }

In [16]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

In [17]:
from sklearn.model_selection import StratifiedKFold
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
predictions = [] # Initialize an array to store predictions
i = 0 
cb_params = {
    'border_count': 233,
    'colsample_bylevel': 0.411551721028093,
    'depth': 8,
    'iterations': 1664,
    'l2_leaf_reg': 1.5162407257384525,
    'learning_rate': 0.01380027964891961,
    'min_child_samples': 218,
    'random_state': 1859,
    'random_strength': 0.4028418793015393,
    'scale_pos_weight': 1.7183429881588008,
    'subsample': 0.996166121986487,
    'verbose': False
}
cb_params_2 = {'objective': 'Logloss',
                                   'eval_metric': "AUC",
                                   'verbose': 0,
                                   'random_state': 42,
                                   'n_estimators': 5000,
                                   'scale_pos_weight': 1.7183429881588008,
                                   'learning_rate': 0.057695680893866355,
                                   'l2_leaf_reg': 8.718243448685213,
                                   'bagging_temperature': 15.939315101482947,
                                   'random_strength': 1.2951116775412417,
                                   'depth': 5,
                                   'min_data_in_leaf': 65 
                                }
            
all_predictions = []
all_truth_values = []
i = 0
training_dataframe = pd.DataFrame()
for fold, (train_index, val_index) in enumerate(skf.split(X_TRAIN, Y_TRAIN)):
    X_train, X_val = X_TRAIN.iloc[train_index], X_TRAIN.iloc[val_index]
    y_train, y_val = Y_TRAIN.iloc[train_index], Y_TRAIN.iloc[val_index]
 
    model =  CatBoostClassifier( **cb_params_3 , cat_features=X_TRAIN.columns.to_list()) 
    model.fit(X_train, y_train)
    fold_predictions = model.predict_proba(X_val)[:,1]
    all_predictions.extend(fold_predictions)
    all_truth_values.extend(y_val)    
   
    print("iteration done ", i)
    predictions.append(model.predict_proba(X_TEST)[:,1]) 
    i = i +1         # Store predictions for the validation fold
  
training_dataframe['cat1'] = all_predictions
training_dataframe['loan_status'] = all_truth_values


iteration done  0
iteration done  1
iteration done  2
iteration done  3
iteration done  4
iteration done  5
iteration done  6
iteration done  7
iteration done  8
iteration done  9


In [18]:

i =0
lgb_predictions = []
all_predictions = []
for fold, (train_index, val_index) in enumerate(skf.split(X_TRAIN_COPY, Y_TRAIN)):
    X_train, X_val = X_TRAIN_COPY.iloc[train_index], X_TRAIN_COPY.iloc[val_index]
    y_train, y_val = Y_TRAIN.iloc[train_index], Y_TRAIN.iloc[val_index]
 
    lgb = LGBMClassifier(**lgb_params)
    lgb.fit(X_train, y_train, categorical_feature= categorical)
    fold_predictions = lgb.predict_proba(X_val)[:,1]
    all_predictions.extend(fold_predictions)
    lgb_predictions.append(lgb.predict_proba(X_TEST_COPY)[:,1]) 
    
    print("iteration done ", i) 
    i = i +1         # Store predictions for the validation fold
    
training_dataframe['lgb1'] = all_predictions

iteration done  0
iteration done  1
iteration done  2
iteration done  3
iteration done  4
iteration done  5
iteration done  6
iteration done  7
iteration done  8
iteration done  9


In [19]:
X_TRAIN_COPY2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 91226 entries, 0 to 32580
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype   
---  ------                      --------------  -----   
 0   person_age                  91226 non-null  int64   
 1   person_income               91226 non-null  int64   
 2   person_home_ownership       91226 non-null  category
 3   person_emp_length           91226 non-null  float64 
 4   loan_intent                 91226 non-null  category
 5   loan_grade                  91226 non-null  category
 6   loan_amnt                   91226 non-null  int64   
 7   loan_int_rate               91226 non-null  float64 
 8   loan_percent_income         91226 non-null  float64 
 9   cb_person_default_on_file   91226 non-null  category
 10  cb_person_cred_hist_length  91226 non-null  int64   
dtypes: category(4), float64(3), int64(4)
memory usage: 5.9 MB


In [20]:
from sklearn.ensemble import HistGradientBoostingClassifier, GradientBoostingClassifier
i =0
gb_predictions = []
all_predictions = []
for fold, (train_index, val_index) in enumerate(skf.split(X_TRAIN_COPY, Y_TRAIN)):
    X_train, X_val = X_TRAIN_COPY.iloc[train_index], X_TRAIN_COPY.iloc[val_index]
    y_train, y_val = Y_TRAIN.iloc[train_index], Y_TRAIN.iloc[val_index]
 
    gb =  GradientBoostingClassifier(**gb_best_params)
    gb.fit(X_train, y_train)
    gb_predictions.append(gb.predict_proba(X_TEST_COPY)[:,1]) 
    fold_predictions = gb.predict_proba(X_val)[:,1]
    all_predictions.extend(fold_predictions)
    print("iteration done ", i) 
    i = i +1         # Store predictions for the validation fold
training_dataframe['gb'] = all_predictions

iteration done  0
iteration done  1
iteration done  2
iteration done  3
iteration done  4
iteration done  5
iteration done  6
iteration done  7
iteration done  8
iteration done  9


In [21]:
from sklearn.ensemble import HistGradientBoostingClassifier
i =0
hgb_predictions = []
all_predictions = []
for fold, (train_index, val_index) in enumerate(skf.split(X_TRAIN_COPY, Y_TRAIN)):
    X_train, X_val = X_TRAIN_COPY.iloc[train_index], X_TRAIN_COPY.iloc[val_index]
    y_train, y_val = Y_TRAIN.iloc[train_index], Y_TRAIN.iloc[val_index]
 
    hgb =  HistGradientBoostingClassifier(**hgb_params)
    hgb.fit(X_train, y_train)
    hgb_predictions.append(hgb.predict_proba(X_TEST_COPY)[:,1]) 
    fold_predictions = hgb.predict_proba(X_val)[:,1]
    all_predictions.extend(fold_predictions)
    print("iteration done ", i) 
    i = i +1         # Store predictions for the validation fold
training_dataframe['hgb'] = all_predictions

iteration done  0
iteration done  1
iteration done  2
iteration done  3
iteration done  4
iteration done  5
iteration done  6
iteration done  7
iteration done  8
iteration done  9


In [22]:
from lightgbm import LGBMClassifier
i =0
lgb_dart_predictions = []
all_predictions = []
for fold, (train_index, val_index) in enumerate(skf.split(X_TRAIN_COPY, Y_TRAIN)):
    X_train, X_val = X_TRAIN_COPY.iloc[train_index], X_TRAIN_COPY.iloc[val_index]
    y_train, y_val = Y_TRAIN.iloc[train_index], Y_TRAIN.iloc[val_index]
 
    lgb_dart = LGBMClassifier(**lgb_dart_params)
    lgb_dart.fit(X_train, y_train, categorical_feature= categorical)
    lgb_dart_predictions.append(lgb_dart.predict_proba(X_TEST_COPY)[:,1]) 
    fold_predictions = lgb_dart.predict_proba(X_val)[:,1]
    all_predictions.extend(fold_predictions)
    print("iteration done ", i) 
    i = i +1         # Store predictions for the validation fold
training_dataframe['lgb2'] = all_predictions

iteration done  0
iteration done  1
iteration done  2
iteration done  3
iteration done  4
iteration done  5
iteration done  6
iteration done  7
iteration done  8
iteration done  9


In [23]:
from xgboost import XGBClassifier
i =0
xgb_predictions = []
all_predictions = []
for fold, (train_index, val_index) in enumerate(skf.split(X_TRAIN_COPY, Y_TRAIN)):
    X_train, X_val = X_TRAIN_COPY.iloc[train_index], X_TRAIN_COPY.iloc[val_index]
    y_train, y_val = Y_TRAIN.iloc[train_index], Y_TRAIN.iloc[val_index]
 
    xgb = XGBClassifier(**xgb_params1)
    xgb.fit(X_train, y_train)
    xgb_predictions.append(xgb.predict_proba(X_TEST_COPY)[:,1]) 
    fold_predictions = xgb.predict_proba(X_val)[:,1]
    all_predictions.extend(fold_predictions)
    print("iteration done ", i) 
    i = i +1         # Store predictions for the validation fold
training_dataframe['xgb'] = all_predictions

iteration done  0
iteration done  1
iteration done  2
iteration done  3
iteration done  4
iteration done  5
iteration done  6
iteration done  7
iteration done  8
iteration done  9


In [24]:
from lightgbm import LGBMClassifier
i =0
lgb_goss_predictions = []
all_predictions = []
for fold, (train_index, val_index) in enumerate(skf.split(X_TRAIN_COPY, Y_TRAIN)):
    X_train, X_val = X_TRAIN_COPY.iloc[train_index], X_TRAIN_COPY.iloc[val_index]
    y_train, y_val = Y_TRAIN.iloc[train_index], Y_TRAIN.iloc[val_index]
 
    lgb_goss = LGBMClassifier(**lgb_goss_params)
    lgb_goss.fit(X_train, y_train, categorical_feature= X_train.columns.to_list())
    
    lgb_goss_predictions.append(lgb_goss.predict_proba(X_TEST_COPY)[:,1]) 
    fold_predictions = lgb_goss.predict_proba(X_val)[:,1]
    all_predictions.extend(fold_predictions)
    print("iteration done ", i) 
    i = i +1         # Store predictions for the validation fold
    
training_dataframe['lgb3'] = all_predictions

iteration done  0
iteration done  1
iteration done  2
iteration done  3
iteration done  4
iteration done  5
iteration done  6
iteration done  7
iteration done  8
iteration done  9


In [25]:
from sklearn.model_selection import StratifiedKFold
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

cat2_predictions = [] # Initialize an array to store predictions
i = 0 
all_predictions = []
for fold, (train_index, val_index) in enumerate(skf.split(X_TRAIN, Y_TRAIN)):
    X_train, X_val = X_TRAIN.iloc[train_index], X_TRAIN.iloc[val_index]
    y_train, y_val = Y_TRAIN.iloc[train_index], Y_TRAIN.iloc[val_index]
 
    model =  CatBoostClassifier( **cb_params_2 , cat_features=X_TRAIN.columns.to_list()) 
     
    model.fit(X_train, y_train)
    fold_predictions = model.predict_proba(X_val)[:,1]
    all_predictions.extend(fold_predictions)
    print("iteration done ", i)
    cat2_predictions.append(model.predict_proba(X_TEST)[:,1]) 
    i = i +1         # Store predictions for the validation fold
    
training_dataframe['cat2'] = all_predictions

iteration done  0
iteration done  1
iteration done  2
iteration done  3
iteration done  4
iteration done  5
iteration done  6
iteration done  7
iteration done  8
iteration done  9


In [26]:
from sklearn.model_selection import StratifiedKFold
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
all_predictions = []
cat3_predictions = [] # Initialize an array to store predictions
i = 0 
for fold, (train_index, val_index) in enumerate(skf.split(X_TRAIN, Y_TRAIN)):
    X_train, X_val = X_TRAIN.iloc[train_index], X_TRAIN.iloc[val_index]
    y_train, y_val = Y_TRAIN.iloc[train_index], Y_TRAIN.iloc[val_index]
 
    model =  CatBoostClassifier( **cb_params_4 , cat_features=X_TRAIN.columns.to_list()) 
     
    model.fit(X_train, y_train)
    
    fold_predictions = model.predict_proba(X_val)[:,1]
    all_predictions.extend(fold_predictions)
    print("iteration done ", i)
    cat3_predictions.append(model.predict_proba(X_TEST)[:,1]) 
    i = i +1         # Store predictions for the validation fold
    
training_dataframe['cat3'] = all_predictions

iteration done  0
iteration done  1
iteration done  2
iteration done  3
iteration done  4
iteration done  5
iteration done  6
iteration done  7
iteration done  8
iteration done  9


In [27]:
from sklearn.linear_model import LogisticRegression

X = training_dataframe
y = X.pop('loan_status')
model = LogisticRegression()
model.fit(X,y)

all_probas = np.stack(predictions)
avg_probas = np.mean(all_probas, axis=0)

all_probas1 = np.stack(lgb_predictions)
avg_probas1 = np.mean(all_probas1, axis=0)

all_probas2 = np.stack(gb_predictions)
avg_probas2 = np.mean(all_probas2, axis=0)

all_probas3 = np.stack(hgb_predictions)
avg_probas3 = np.mean(all_probas3, axis=0)

all_probas4 = np.stack(lgb_dart_predictions)
avg_probas4 = np.mean(all_probas4, axis=0)

all_probas5 = np.stack(lgb_goss_predictions)
avg_probas5 = np.mean(all_probas5, axis=0)

all_probas6 = np.stack(xgb_predictions)
avg_probas6 = np.mean(all_probas6, axis=0)

all_probas7 = np.stack(cat2_predictions)
avg_probas7 = np.mean(all_probas7, axis=0)

all_probas8 = np.stack(cat3_predictions)
avg_probas8 = np.mean(all_probas8, axis=0)

final_df = pd.DataFrame()
final_df['cat1'] = avg_probas
final_df['lgb1'] = avg_probas1
final_df['gb'] = avg_probas2
final_df['hgb'] = avg_probas3
final_df['lgb2'] = avg_probas4
final_df['xgb'] = avg_probas6
final_df['lgb3'] = avg_probas5
final_df['cat2'] = avg_probas
final_df['cat3'] = avg_probas

total_probas = model.predict_proba(final_df)[:,1] 

In [28]:
probs = total_probas
Result = pd.DataFrame()
Result['id']=id
if len(probs.shape) >1:
    Result['loan_status'] = probs[:,1]
else :
    Result['loan_status'] = probs

In [29]:
Result.to_csv("submission.csv",index=False)

In [30]:
pd.__version__

'2.2.3'

In [31]:
import xgboost as xgb

xgb.__version__

'2.0.3'

In [32]:
a, b = 3, 4

In [33]:
a

3